# Pull the LightVision repository

In [1]:
try:
    import google.colab
    !git clone https://github.com/erenyavuz02/LightVision.git
    print("Repository cloned in Google Colab")
except ImportError:
    print("Running locally - repository clone skipped")

Running locally - repository clone skipped


# Set the config manager to use the local config file

In [2]:
import os
import sys

# Add project root to path
try:
    # Try to detect if running in Google Colab
    import google.colab
    # If running in Google Colab, you need to manually set the project root
    # Update this path to match where you cloned/uploaded your project in Colab
    project_root = '/content/LightVision'  # Adjust this path as needed for your Colab setup
    print("Running in Google Colab")
except ImportError:
    # Running locally
    script_dir = os.path.dirname(os.path.abspath("__file__"))
    project_root = os.path.abspath(os.path.join(script_dir, '.'))
    print("Running locally")

if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(f"Project root: {project_root}")

# Import ConfigManager
from utils.config import ConfigManager


Running locally
Project root: /Users/erenyavuz/Desktop/KU/25 Spring/COMP447/Project/Repo/FlightVision


In [3]:
config = ConfigManager(config_path = os.path.join(project_root, 'config/config.yaml'))

# set the project root in the config project: 
config.update('project.root', project_root)

# Download the flickr dataset

In [4]:
# Download Flickr8k dataset
from functions.dataset import DatasetDownloader
from pathlib import Path

datasetDownloader = DatasetDownloader(config)

# Download dataset - removed extra config parameter
if datasetDownloader.download_dataset(verbose=True):
    # Verify dataset
    images_dir = Path(config.get('project.root')) / 'data' / 'Images'
    image_count = len(list(images_dir.glob('*.jpg')))
    print(f"✅ Flickr8k Dataset: {image_count:,} images ready")
else:
    print("❌ Dataset download failed")
    

Starting download process for Flickr8k...
Dataset already exists: 8091 files found
✅ Flickr8k Dataset: 8,091 images ready


# Download the base model

In [5]:
# Download base MobileCLIP model
from functions.model import download_base_model

if download_base_model(config, verbose=True):
    print("🚀 Base model ready for training!")
else:
    print("❌ Base model download failed")

/opt/anaconda3/lib/python3.12/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


✅ Base model already exists.
🚀 Base model ready for training!


# Load the base model

In [6]:
from functions.model import load_model

base_model, preprocess, tokenizer = load_model(config, verbose=True)

💻 CUDA not available, using CPU
Loading mobileclip_s0 model...
Loading from checkpoint: /Users/erenyavuz/Desktop/KU/25 Spring/COMP447/Project/Repo/FlightVision/checkpoints/mobileclip_s0.pt
✅ Model loaded successfully on cpu


# Test Custom Dataset with Train/Test Split

Test the CustomDataset class to ensure proper train/test splitting without data leakage.

In [7]:
# Test CustomDataset class with single instance
from functions.dataset import CustomDataset

# set the captions file here
config.update('dataset.captions_file', 'all_captions.json')  # Use all captions for testing

# Create a single dataset instance
print("Creating dataset instance...")
dataset = CustomDataset(config, test_ratio=0.125, transform=preprocess)


Creating dataset instance...


# Evaluate the base model

In [9]:
import importlib
from functions.evaluate import evaluate_dataset

import functions.evaluate
importlib.reload(functions.evaluate)

<module 'functions.evaluate' from '/Users/erenyavuz/Desktop/KU/25 Spring/COMP447/Project/Repo/FlightVision/functions/evaluate.py'>

In [10]:
from functions.evaluate import evaluate_dataset

# Evaluate the base model
print("Evaluating base model performance...")

# Run evaluation
evaluation_results = evaluate_dataset(
    model=base_model,
    testDataset=dataset,
    config=config,
    tokenizer = tokenizer,
    k_values=[1, 5, 10, 20],  # You can customize k values
    force_rebuild_index=True,  # Set to True to force rebuild FAISS index
    verbose=True,
    only_test=True  # Set to True to evaluate only on the test set
)

# The results will be automatically saved and printed
print("Evaluation complete!")

Evaluating base model performance...
Starting dataset evaluation...
Model: mobileclip_s0
Device: cpu
Dataset split: 7080 train, 1011 test

Evaluating test split
Building new FAISS index...
Extracting image embeddings for test split...
Processing batch 1/32
Processing batch 11/32
Processing batch 21/32
Processing batch 31/32
Extracted 1011 image embeddings with dimension 512
Building FAISS index...
FAISS index built with 1011 vectors
Index saved to /Users/erenyavuz/Desktop/KU/25 Spring/COMP447/Project/Repo/FlightVision/data/faiss_cache/mobileclip_s0_test_index.faiss
Metadata saved to /Users/erenyavuz/Desktop/KU/25 Spring/COMP447/Project/Repo/FlightVision/data/faiss_cache/mobileclip_s0_test_metadata.pkl
Evaluating retrieval on test split...
Processing 1011 queries...
Processing query 1/1011
Processing 3 caption subsections
  Caption 1: In the scene, we see two young men standing in an indoor parking area or garage-like setting with th...
  Caption 2: The surroundings are furnished with v

# Model Training with Custom Dataset

Now we'll train our model using the custom dataset with positional embedding modifications. This will:
1. Apply the positional embedding strategy to improve text encoding
2. Train the modified model on our custom dataset
3. Track training progress with real-time visualizations

In [ ]:
# Import training functions
from functions.train import train_model

# Set training parameters
training_config = {
    'num_epochs': 15,
    'batch_size': 16,  
    'learning_rate': 1e-4,
}

print("Starting training process...")
print(f"Configuration: {training_config}")
print("\n" + "="*60)

In [ ]:
# Train the model with positional embedding modifications
trained_model = train_model(
    model= base_model,  # Your original CLIP model
    config=config,  # Your existing config
    dataset = dataset,
    num_epochs=training_config['num_epochs'],
    batch_size=training_config['batch_size'],
    learning_rate=training_config['learning_rate']
)

print("\n🎉 Training completed successfully!")
print("The model has been trained with:")
print("✓ Modified positional embeddings")
print("✓ Contrastive learning approach")

In [ ]:
# Load the best model for further evaluation
import torch

# Load the best saved model
checkpoint = torch.load('best_model.pth')
base_model.load_state_dict(checkpoint['model_state_dict'])

print("Best model loaded!")
print(f"Best validation loss achieved: {checkpoint['val_loss']:.4f}")
print(f"Best model was saved at epoch: {checkpoint['epoch'] + 1}")

# You can now use this trained model for inference
#base_model.eval()

# Evaluate the trained model

In [ ]:
# evaluate the trained model
evaluation_results = evaluate_dataset(
    model=base_model,
    testDataset=dataset,
    config=config,
    k_values=[1, 5, 10, 20],  # You can customize k values
    force_rebuild_index=True,  # Set to True to force rebuild FAISS index
    verbose=True,
    only_test=True  # Set to True to evaluate only on the test set
)
# The results will be automatically saved and printed
print("Evaluation of the trained model complete!")

